1. Check out this official repository with many examples of Keras implementations of various sorts of
deep neural networks here. We recommend cloning this repository and trying to get some of these
examples running on your system (or Colab/DeepNote). In particular, experiment with mnist mlp.py
and mnist cnn.py scripts which show you how to build simple neural networks for the MNIST dataset
(useful for the next task

Next, take the two well-known datasets: Fashion MNIST (introduced in Ch 10, p. 298) and CIFAR-10.
The first dataset contains 2D (grayscale) images of size 28x28, split into 10 categories; 60,000 images
for training and 10,000 for testing, while the latter contains 32x32x3 RGB images (50,000/10,000
train/test). Apply two reference networks on the fashion MNIST datase

(a) A multi-layer perceptron described in detail in Ch. 10, pp. 299-307

initializations, activations, optimizers (and
their hyperparameters), regularizations (L1, L2, Dropout, no Dropout). You may also experiment
with changing the architecture of both networks: adding/removing layers, number of convolutional
filters, their sizes, etc.

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2025-10-16 16:20:18.108276: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-16 16:20:18.155129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-16 16:20:18.171526: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-16 16:20:18.175299: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-16 16:20:18.210989: I tensorflow/core/platform/cpu_feature_guar

Num GPUs Available:  1


In [ ]:
# Use TensorFlow's bundled Keras to ensure compatibility with GPUs
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.preprocessing import StandardScaler

# Try to enable memory growth for all GPUs so TF doesn't reserve all GPU memory upfront
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print('Enabled memory growth for GPUs:', gpus)
    except Exception as e:
        print('Could not set memory growth:', e)
else:
    print('No GPU devices found by TensorFlow')


# tf.config.optimizer.set_jit(False)

# jit_status = tf.config.optimizer.get_jit()
# print("JIT enabled:", bool(jit_status))
fashion_mnist= keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print(X_train_full.shape, y_train_full.shape)



X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)







#dropout 0.1 = 0.8635, 0 droput= 0.8744, 0.02 droput = 0.8754, 0.05 = 0.8766,
#leaky = 0.8770, leaky+0.1 dropout=0.8798
#l 1 regularization = 0.8552, l2 regularization  0.01 =0.8762 0.001 = 0.8798
#extra layer 0.8800
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(400, activation='leaky_relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(200, activation="leaky_relu"),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(100, activation="leaky_relu"),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(50, activation="leaky_relu"),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(10, activation="softmax")
])
early_stopping = keras.callbacks.EarlyStopping(
    patience=6,
    restore_best_weights=True
)

model.summary()


model.compile(loss="sparse_categorical_crossentropy",
optimizer=keras.optimizers.SGD(learning_rate=2e-3),
metrics=["accuracy",
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ])
model.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping]
)
#evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
#0.8256000280380249
#use only a small subset of the training data
# X_train = X_train[:20000]
# y_train = y_train[:20000]
# X_valid = X_valid[:1000]
# y_valid = y_valid[:1000]


Enabled memory growth for GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
(60000, 28, 28) (60000,)
(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)
(60000, 28, 28) (60000,)
(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)


/home/ollie/anaconda3/envs/idl/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-10-16 16:21:01.287872: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6132 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 400)            │       314,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │        80,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 419,860 (1.60 MB)

...

Epoch 1/30


2025-10-16 16:21:02.376415: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-16 16:21:02.396716: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101


1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.4007 - loss: 2.2921 - val_accuracy: 0.7436 - val_loss: 1.2644
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.4007 - loss: 2.2921 - val_accuracy: 0.7436 - val_loss: 1.2644
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 1s 588us/step - accuracy: 0.7150 - loss: 1.3188 - val_accuracy: 0.8018 - val_loss: 1.1132
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 1s 588us/step - accuracy: 0.7150 - loss: 1.3188 - val_accuracy: 0.8018 - val_loss: 1.1132
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 1s 586us/step - accuracy: 0.7667 - loss: 1.1796 - val_accuracy: 0.8178 - val_loss: 1.0499
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 1s 586us/step - accuracy: 0.7667 - loss: 1.1796 - val_accuracy: 0.8178 - val_loss: 1.0499
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 1s 590us/step - accuracy: 0.7904 - loss: 1.1091 - val_accuracy: 0.8266 - val_loss: 1.0066
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 1s 590us/step - accuracy: 0.7904 - loss: 1.1091 -

/home/ollie/anaconda3/envs/idl/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.1655 - loss: 2.2565 - val_accuracy: 0.6266 - val_loss: 1.2637
Epoch 2/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.1655 - loss: 2.2565 - val_accuracy: 0.6266 - val_loss: 1.2637
Epoch 2/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.4734 - loss: 1.4490 - val_accuracy: 0.7038 - val_loss: 0.8349
Epoch 3/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.4734 - loss: 1.4490 - val_accuracy: 0.7038 - val_loss: 0.8349
Epoch 3/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5947 - loss: 1.1040 - val_accuracy: 0.7518 - val_loss: 0.7182
Epoch 4/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5947 - loss: 1.1040 - val_accuracy: 0.7518 - val_loss: 0.7182
Epoch 4/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6432 - loss: 0.9762 - val_accuracy: 0.7402 - val_loss: 0.6787
Epoch 5/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6432 - loss: 0.9762 - 

In [ ]:
max_pool = keras.layers.MaxPool2D(pool_size=2)
# avg_pool = keras.layers.AveragePooling2D(pool_size=2)
model = keras.models.Sequential([
keras.layers.Conv2D(64, 7, activation="leaky_relu", padding="same",
input_shape=[28, 28, 1]),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(2),
keras.layers.Conv2D(128, 3, activation="leaky_relu", padding="same"),
keras.layers.BatchNormalization(),
keras.layers.Conv2D(128, 3, activation="leaky_relu", padding="same"),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(2),
# keras.layers.Dropout(0.12,
keras.layers.Conv2D(256, 3, activation="leaky_relu", padding="same"),
keras.layers.BatchNormalization(),
keras.layers.Conv2D(256, 3, activation="leaky_relu", padding="same"),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(2),
keras.layers.Flatten(),
keras.layers.Dense(128, activation="leaky_relu"),
keras.layers.Dropout(0.5),
keras.layers.Dense(64, activation="leaky_relu"),
keras.layers.Dropout(0.5),
keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
optimizer=keras.optimizers.SGD(learning_rate=2e-3),
metrics=["accuracy",
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ])
model.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    # callbacks=[early_stopping]
)
#evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
#base:0.8420000076293945
#leaky: 0.8525000214576721
#leaky + L2regularization: 0.8472999930381775
#leaky + batch normalization: 0.8978000283241272

(print(tf.__version__))
# metrics=["accuracy"])

Epoch 1/30


/home/ollie/anaconda3/envs/idl/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.6408 - loss: 1.1211 - val_accuracy: 0.8526 - val_loss: 0.4031
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.6408 - loss: 1.1211 - val_accuracy: 0.8526 - val_loss: 0.4031
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8070 - loss: 0.5666 - val_accuracy: 0.8864 - val_loss: 0.3262
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8070 - loss: 0.5666 - val_accuracy: 0.8864 - val_loss: 0.3262
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8406 - loss: 0.4669 - val_accuracy: 0.8906 - val_loss: 0.3024
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8406 - loss: 0.4669 - val_accuracy: 0.8906 - val_loss: 0.3024
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8568 - loss: 0.4160 - val_accuracy: 0.9020 - val_loss: 0.2791
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8568 - loss: 0.4160 - val_accurac

In [9]:
X_test = X_test / 255.0
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8983 - loss: 0.3457
Test accuracy: 0.8978000283241272
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8983 - loss: 0.3457
Test accuracy: 0.8978000283241272
